In [ ]:
#!/usr/bin/python
# -*- coding:utf-8 -*-
#standard python packages
import numpy as np
import math
import matplotlib.pyplot as plt
#this program apparently needs more memory than my laptop have to be able to run files with sizes about 3.4 GB
##################################################
##changeable things, for now only the first part of getting the total maximum will work if several files are wanted
##for a single file all cells work

outname = "text_files/" + "simu.2017-07-29-00h41m58s_max" + ".txt"    #in case for saving
#namelist = ["BlueJet1_128GTU.txt", "BlueJet2_128GTU.txt", "BlueJet3_128GTU.txt", "BlueJet5_128GTU.txt", "BlueJet6_128GTU.txt", "BlueJet7_128GTU.txt", "Elves1_128GTU.txt", "Elves2_128GTU.txt", "Sprite1_128GTU.txt", "Sprite2_128GTU.txt", "Meteor_1_L2.txt", "Meteor_2_L2.txt"]    #in case of wanting to go through multiple files
saving = 0    #0 = saves to file, anything else will only run
show = 1    #0 = shows plot, anything else will only run

##below is for testing, create outname file before saving or the saving part needs to be done twice since the first time only an empty file is created
#outname = "text_files/" +  "test_max.txt"
namelist = ["simu.2017-07-29-00h41m58s"]
#saving = 0    #0 = saves to file, anything else will only run
#show = 0    #0 = shows plot, anything else will only run

##################################################

maximum_collection = []
total_ind = 0
for name in namelist:
    total_list = []    #list of the maxmimum number for each timestep in the file
    tempname = "text_files/" + name + ".txt"
    with open(tempname) as f:
        data = []
        for line in f:
            line = line.split()
            if line:
                line = [int(i) for i in line]
                data.append(line)    #making the text file into a list

        length = np.shape(data)[0]    #number of rows in the text file
        ind = range(0, length, 49)    #list of numbers with the indices in the list data indicating change in timestep, [0, 49, 98, 147....]
        total_ind = total_ind + len(ind)
        nz = int(len(ind))    #nz is the total number of timesteps in the current textfile
        counts = np.zeros((nz, 2304))    #list of nz lists each with 2304 zeros corresponding to each pixel
        for iz in range(nz):
            counts[iz][:] = np.concatenate((data[ind[iz] + 1 : ind[iz] + 49]), axis = 0)    #filling the nz lists with the data

    for maximum in counts:
        total_list.append(max(maximum))

    maximum_collection.append(name + "  " + str(max(total_list)))


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records


Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 851, in emit
Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 851, in emit
Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 851, in emit
Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 851, in emit
Error in sys.excepthook:
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 162, in excepthook
    traceback.print_exception(etype, evalue, tb, file=sys.__stderr__)
  File "/usr/lib/python2.7/traceback.py", line 125, in print_exception
    print_tb(tb, limit, file)
  File "/usr/lib/python2.7/traceback.py", line 69, in print_tb
    line = linecache.getline(filename, lineno, f.f_globals)
  File "/usr/lib/python2.7/linecache.py", line 14, in getline
    lines = getlines(filename, module_globals)
  File "/usr/lib/python2.7/lineca

In [4]:
#for saving the maximum for each text file opened
if saving == 0:
    thefile = open(outname, 'w')
    for line in maximum_collection:    #goes through the total_list and writes each line in the text file
        thefile.write(line + "\n")

In [2]:
##below is to find the pixel with the maximum, does not work for multiple files at the same time since total_list gets overwritten
## in case multiple files is wanted, run make_composite_texts first on the wanted files and then this on the composite text
pixel = []
x_max = 0
y_max = 0
for a in range(nz):
    w = counts[a].reshape(48, 48)    #this makes the array of 2304 numbers in a 48x48 shape
    temp_y,temp_x = np.where(w==total_list[a])    #this gives the pixels which are maximum
    y_max_temp,x_max_temp = np.where(w==max(total_list))    #this gives the pixels which are maximum
    if len(y_max_temp) > 0:
        y_max = y_max_temp
        x_max = x_max_temp
    for q in range(len(temp_x)):    #loop to make the pixel into a format that is easier to read
        temp = str(a+1) + "; (" + str(temp_x[q]) + "," + str(47-temp_y[q]) + "); " + str(total_list[a])    #to make it look good
        pixel.append(temp)    #saves in the following format: "frame#; (x,y); maximum"

##below is for printing the pixel
#for a in range(len(pixel)):
#    print pixel[a]


In [40]:
#for saving the maximum pixel for each frame (the file will be very long if the files are unintegrated)
if saving == 0:
    thefile = open("text_files/simu.2017-07-29-00h41m58s_pixel_max.txt", 'w')
    for line in pixel:    #goes through the total_list and writes each line in the text file
        thefile.write(line + "\n")


In [3]:
#below is for making a plot over how the maximum pixel changes over all timestamps
indexnumber = 48*y_max[0] + x_max[0]    #the index in counts for the maximum pixel
#print "Index number in counts for the maximum pixel is " + str(indexnumber)
max_pixel_change = []
plot_savename = "plots/" + namelist[0].strip(".txt")    #this will be the name the plot is saved as, no file ending
for a in counts:    #makes a list of all timestamps for the maximum pixel
    max_pixel_change.append(a[indexnumber])
x_range = range(1,len(max_pixel_change)+1)
fig = plt.figure()
ax = plt.gca()
plt.plot(x_range, max_pixel_change)
plt.title('Filename = ' + namelist[0])
plt.ylabel('Counts')
plt.xlabel('Timestamp')
if show == 0:    #remember that the python program will not continue until the figure is closed
    plt.show()

In [42]:
#for saving how the maximum pixel changes over each timestamp
if saving == 0:
    fig.savefig(plot_savename + "_max.eps")
